In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset # 텐서데이터셋
from torch.utils.data import DataLoader # 데이터로더
import pandas as pd
import numpy as np

C:\Anaconda3\envs\py39tf2\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
torch.manual_seed(1)

In [3]:
df = pd.read_csv('gpascore.csv')
df

,admit,gre,gpa,rank
0,0,380.0,3.21,3
1,1,660.0,3.67,3
2,1,800.0,4.00,1
3,1,640.0,3.19,4
4,0,520.0,2.33,4
...,...,...,...,...
421,1,750.0,3.44,4
422,1,710.0,3.66,4
423,0,450.0,3.25,4
424,1,760.0,3.76,2


In [4]:
df.iloc[19,:]

admit    1.00
gre       NaN
gpa      3.81
rank     1.00
Name: 19, dtype: float64

In [5]:
df.drop(19, inplace=True)

In [6]:
df.iloc[19,:]

admit      0.00
gre      500.00
gpa        3.17
rank       3.00
Name: 20, dtype: float64

In [7]:
from sklearn.preprocessing import MinMaxScaler

In [8]:
scaler = MinMaxScaler()

In [9]:
scaler.fit(df)

MinMaxScaler()

In [10]:
df_scaled = scaler.transform(df)

In [11]:
df_scaled

array([[0.        , 0.27586207, 0.57978723, 0.66666667],
       [1.        , 0.75862069, 0.82446809, 0.66666667],
       [1.        , 1.        , 1.        , 0.        ],
       ...,
       [0.        , 0.39655172, 0.60106383, 1.        ],
       [1.        , 0.93103448, 0.87234043, 0.33333333],
       [1.        , 0.84482759, 0.90425532, 0.66666667]])

In [12]:
x_train = np.array(df.iloc[:,1:4])
y_train = np.array(df.iloc[:,0])

In [13]:
x_train = torch.FloatTensor(x_train)
y_train = torch.FloatTensor(y_train)
y_train = y_train.reshape(425,1)
print(x_train.shape)
print(y_train.shape)

torch.Size([425, 3])
torch.Size([425, 1])


In [41]:
# 모델 초기화
W = torch.zeros((3, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

In [42]:
# optimizer 설정
optimizer = optim.Adam([W, b], lr=0.01)

In [48]:
nb_epochs = 5000
for epoch in range(nb_epochs + 1):

    # Cost 계산
    hypothesis = torch.sigmoid(x_train.matmul(W) + b)
    cost = F.mse_loss(hypothesis, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/5000 Cost: 0.152397
Epoch  100/5000 Cost: 0.151441
Epoch  200/5000 Cost: 0.151329
Epoch  300/5000 Cost: 0.151227
Epoch  400/5000 Cost: 0.151122
Epoch  500/5000 Cost: 0.151015
Epoch  600/5000 Cost: 0.150907
Epoch  700/5000 Cost: 0.150796
Epoch  800/5000 Cost: 0.150684
Epoch  900/5000 Cost: 0.150569
Epoch 1000/5000 Cost: 0.150453
Epoch 1100/5000 Cost: 0.150335
Epoch 1200/5000 Cost: 0.150215
Epoch 1300/5000 Cost: 0.150094
Epoch 1400/5000 Cost: 0.149972
Epoch 1500/5000 Cost: 0.149848
Epoch 1600/5000 Cost: 0.149724
Epoch 1700/5000 Cost: 0.149599
Epoch 1800/5000 Cost: 0.149779
Epoch 1900/5000 Cost: 0.149838
Epoch 2000/5000 Cost: 0.150075
Epoch 2100/5000 Cost: 0.152882
Epoch 2200/5000 Cost: 0.149997
Epoch 2300/5000 Cost: 0.149295
Epoch 2400/5000 Cost: 0.150736
Epoch 2500/5000 Cost: 0.148804
Epoch 2600/5000 Cost: 0.148623
Epoch 2700/5000 Cost: 0.148525
Epoch 2800/5000 Cost: 0.148744
Epoch 2900/5000 Cost: 0.149602
Epoch 3000/5000 Cost: 0.148295
Epoch 3100/5000 Cost: 0.152185
Epoch 32

In [49]:
hypothesis = torch.sigmoid(x_train.matmul(W) + b)
print(hypothesis[:5])

tensor([[0.0671],
        [0.7210],
        [0.9501],
        [0.3361],
        [0.0129]], grad_fn=<SliceBackward0>)


In [50]:
prediction = hypothesis >= torch.FloatTensor([0.5])
print(prediction[:5])

tensor([[False],
        [ True],
        [ True],
        [False],
        [False]])


In [51]:
correct_prediction = prediction.float() == y_train
print(correct_prediction[:5])

tensor([[ True],
        [ True],
        [ True],
        [False],
        [ True]])


In [52]:
accuracy = correct_prediction.sum().item() / len(correct_prediction)
print('The model has an accuracy of {:2.2f}% for the training set.'.format(accuracy * 100))

The model has an accuracy of 80.71% for the training set.
